In [2]:
# !pip install PyMuPDF

In [3]:
# !pip install tk

In [4]:
# !pip install py-pdf-parser

In [5]:
# import fitz 
# https://www.geeksforgeeks.org/extract-text-from-pdf-file-using-python/

In [6]:
# doc = fitz.open("think_bayes_trunc.pdf")
# type(doc)

In [7]:
# text = ''
# # for page in doc:
#     text = page.get_text()
    

In [8]:
# text #very long outputvv

# trying out PyPDF Parser to collect headings

In [9]:
# !pip install py-pdf-parser[dev]

In [10]:
# import numpy as np

In [11]:
# from py_pdf_parser.loaders import load_file

In [12]:
# from py_pdf_parser.visualise import visualise
# this is bogus bc i need to install the dev vers of py_pdf_parser which doesnt exist

In [13]:
# pdfParse = load_file("think_bayes_trunc.pdf")

In [14]:
# for elem in pdfParse.elements:
#     print(elem.font)

### it works and i get fonts, but without the visualizer, I can't use this library to its full capabilities

# take 2: rather than parsing the data, can i just give the model a blob?

In [15]:
import pandas as pd
import fitz 
# https://www.geeksforgeeks.org/extract-text-from-pdf-file-using-python/

In [16]:
def make_ds(data, doc_title, count, sample_size, document):
    for page in document:
        # print("len" + str(text_df.shape))
        text = page.get_text()
        if len(text) > sample_size:
            add_in_sections(data, doc_title, text, sample_size, count)
            count +=1
        else:
            data.loc[data.shape[0]-1] = [doc_title, count, text, 1.0]
            count +=1 
    return data
        

In [17]:
def add_in_sections(data, doc_title, text, sample_size, count):
    if len(text) != 0:
        if sample_size < len(text):
            data.loc[data.shape[0]-1] = [doc_title, count, text[:len(text)], 1.0]
            count +=1
            add_in_sections(data, doc_title, text[sample_size:], sample_size, count)
        else:
            data.loc[data.shape[0]-1] = [doc_title, count, text[:sample_size], 1.0]

In [18]:
doc_train = fitz.open("think_bayes_train.pdf")
text_df = pd.DataFrame(columns=["title", "count", "body", "labels"])

In [19]:
text_df = make_ds(text_df, "Thinking Bayes: Bayesian Statistics in Python", 0, 520, doc_train)
#520 max sample length for roberta

In [20]:
text_df.head()

,title,count,body,labels
-1,Thinking Bayes: Bayesian Statistics in Python,0,Allen B. Downey\nSecond \nEdition\nThink \n B...,1.0
0,Thinking Bayes: Bayesian Statistics in Python,1,,1.0
1,Thinking Bayes: Bayesian Statistics in Python,2,Allen B. Downey\nThink Bayes\nBayesian Statist...,1.0
2,Thinking Bayes: Bayesian Statistics in Python,3,978-1-492-08946-9\n[LSI]\nThink Bayes\nby Alle...,1.0
3,Thinking Bayes: Bayesian Statistics in Python,4,tions Editor: Jessica Haberman\nDevelopment Ed...,1.0


## start tokenizing

In [21]:
!pip install datasets

In [22]:
!pip install transformers

In [24]:
!pip install sentencepiece

In [25]:
from datasets import Dataset,DatasetDict
#turn our pandas dataframe into a huggingface dataset to do huggingface things with it
text_ds = Dataset.from_pandas(text_df) 

In [26]:
text_ds

Dataset({
    features: ['title', 'count', 'body', 'labels', '__index_level_0__'],
    num_rows: 864
})

In [27]:
model_nm = 'microsoft/deberta-v3-small'

In [28]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

2023-01-14 15:18:59.291570: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/Users/abinayadinesh/opt/anaconda3/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have

In [29]:
type(tokz)

transformers.models.deberta_v2.tokenization_deberta_v2_fast.DebertaV2TokenizerFast

In [30]:
# tokenize our inputs
def tok_func(x):
    return tokz(x['body'])

In [31]:
tok_text_ds = text_ds.map(tok_func, batched=True)
# ds = Dataset.from_pandas(df)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [32]:
type(tok_text_ds)

datasets.arrow_dataset.Dataset

In [33]:
dds = tok_text_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['title', 'count', 'body', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 648
    })
    test: Dataset({
        features: ['title', 'count', 'body', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 216
    })
})

### load test set

In [34]:
doc_test = fitz.open("think_bayes_test.pdf")

In [35]:
eval_df = pd.DataFrame(columns=["title", "count", "body", "labels"])

In [36]:
eval_df = make_ds(eval_df, "Thinking Bayes: Bayesian Statistics in Python", 1245, 520, doc_test)

In [37]:
eval_df.head()

,title,count,body,labels
-1,Thinking Bayes: Bayesian Statistics in Python,1245,CHAPTER 15\nMark and Recapture\nThis chapter i...,1.0
0,Thinking Bayes: Bayesian Statistics in Python,1246,ll work with models that have three parameters...,1.0
1,Thinking Bayes: Bayesian Statistics in Python,1247,"ng the hair samples, the researchers\nuse DNA ...",1.0
2,Thinking Bayes: Bayesian Statistics in Python,1246,To estimate the population of bears from this ...,1.0
3,Thinking Bayes: Bayesian Statistics in Python,1247,0 bears have been identified. During the secon...,1.0


In [38]:
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [39]:
eval_ds

Dataset({
    features: ['title', 'count', 'body', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 381
})

# Train Our Model

In [40]:
!pip install torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [41]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [42]:
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification

In [43]:
bs = 128
epochs = 4

In [44]:
lr = 8e-5

In [45]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine',
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')
# warmup ratio is the same as fit_one_cycle in fast ai

In [46]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'], compute_metrics=corr_d)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: count, __index_level_0__, title, body. If count, __index_level_0__, title, body are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
/Users/abinayadinesh/opt/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 648
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 24
  Number of trainable parameters = 141895681


In [ ]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds